In [2]:
import pandas as pd

In [3]:
en_items = pd.read_csv('odeuropa_enriched_april30th-edited.csv', index_col='item_id')

en_items['emotion_llama'] = None

en_items.head()

,Unnamed: 0,id,graph,label,source_id,source_label,source_url,source_date,source_language,relevantExcerpt,...,words,year,sentiment,excerpt_value_cleaned,source_label_cleaned,geoname_id,countries,final_country_code,final_country_name,emotion_llama
item_id,,,,,,,,,,,,,,,,,,,,,
0,0,http://data.odeuropa.eu/smell/a09e1c3c-f023-5e...,http://data.odeuropa.eu/british-library,smell,http://data.odeuropa.eu/source/56c5cda7-750c-5...,Geschichte der Stadt und Bäder zu Baden [in C...,NaN,1880,en,http://data.odeuropa.eu/source/56c5cda7-750c-5...,...,"['sniffing', 'the party', 'betimes']",1880,0.088636,34 owd siah waved termed thick knotty meet meb...,geschichte der stadt und bäder zu baden canto...,10179318,['IR'],IR,Iran,None
1,1,http://data.odeuropa.eu/smell/a09e1c3c-f023-5e...,http://data.odeuropa.eu/british-library,smell,http://data.odeuropa.eu/source/56c5cda7-750c-5...,Geschichte der Stadt und Bäder zu Baden [in C...,NaN,1880,en,http://data.odeuropa.eu/source/56c5cda7-750c-5...,...,"['Bartlett', 'perfume', 'scent', 'very odorous...",1880,-0.071652,179 arm linked lucy phœbe louise following lit...,geschichte der stadt und bäder zu baden canto...,10179318,['IR'],IR,Iran,None
2,2,http://data.odeuropa.eu/smell/a09e1c3c-f023-5e...,http://data.odeuropa.eu/british-library,smell,http://data.odeuropa.eu/source/56c5cda7-750c-5...,Geschichte der Stadt und Bäder zu Baden [in C...,NaN,1880,en,http://data.odeuropa.eu/source/56c5cda7-750c-5...,...,"['stench', 'noysome', 'his', 'crawling']",1880,-0.062500,104 picture eccho forth grones hollow truncke ...,geschichte der stadt und bäder zu baden canto...,10179318,['IR'],IR,Iran,None
3,3,http://data.odeuropa.eu/smell/a09e1c3c-f023-5e...,http://data.odeuropa.eu/british-library,smell,http://data.odeuropa.eu/source/56c5cda7-750c-5...,Geschichte der Stadt und Bäder zu Baden [in C...,NaN,1880,en,http://data.odeuropa.eu/source/56c5cda7-750c-5...,...,['fragrant'],1880,0.083333,nothing perhaps sends quicksilver sphits highe...,geschichte der stadt und bäder zu baden canto...,10179318,['IR'],IR,Iran,None
4,4,http://data.odeuropa.eu/smell/a09e1c3c-f023-5e...,http://data.odeuropa.eu/british-library,smell,http://data.odeuropa.eu/source/56c5cda7-750c-5...,Geschichte der Stadt und Bäder zu Baden [in C...,NaN,1880,en,http://data.odeuropa.eu/source/56c5cda7-750c-5...,...,"['scent', 'thy', 'so much sharper']",1880,0.202273,open space sent forth shout joy echoed mountai...,geschichte der stadt und bäder zu baden canto...,10179318,['IR'],IR,Iran,None


In [4]:
import pandas as pd
from tqdm import tqdm
import requests
import re

def detect_emotion(text):
    """Send text to the local LLaMA model and return the detected emotion based on Plutchik's wheel."""
    emotions = ["joy", "trust", "fear", "surprise", "sadness", "disgust", "anger", "anticipation"]
    pattern = re.compile(r'\b(' + '|'.join(emotions) + r')\b', re.IGNORECASE)
    
    data = {
        "model": "llama3",
        "prompt": f"You are trained to identify emotions based on Plutchiks wheel. You will only answer with the primary emotion and not a single word more. Your only options are: joy, trust, fear, surprise, sadness, disgust, anger and anticipation. What is the primary emotion in this text based on Plutchiks wheel of emotions: {text}",
        "stream": False
    }
    
    try:
        response = requests.post('http://localhost:11434/api/generate', json=data)
        response_data = response.json()
        response_text = response_data["response"].strip().lower()
        
        match = pattern.search(response_text)
        if match:
            return match.group(0)
        else:
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

for index, row in tqdm(en_items.iloc[:10].iterrows(), total=10):
    emotion = detect_emotion(row['excerpt_value_cleaned'])
    en_items.at[index, 'emotion_llama'] = emotion

en_items[['excerpt_value_cleaned','emotion_llama']]


100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:14<00:00,  1.44s/it]


,excerpt_value_cleaned,emotion_llama
item_id,,
0,34 owd siah waved termed thick knotty meet meb...,fear
1,179 arm linked lucy phœbe louise following lit...,joy
2,104 picture eccho forth grones hollow truncke ...,fear
3,nothing perhaps sends quicksilver sphits highe...,anticipation
4,open space sent forth shout joy echoed mountai...,joy
...,...,...
78868,ever sweeter bloomingfieldsandverdantmeadows o...,None
78869,â   5th dying patriarch isaac pleased smell ...,None
78870,mercy god ex hausted sin may appear trifling o...,None


In [5]:
for index, row in tqdm(en_items.iloc[10:20010].iterrows(), total=20000):
    emotion = detect_emotion(row['excerpt_value_cleaned'])
    en_items.at[index, 'emotion_llama'] = emotion

100%|███████████████████████████████████████████████████████████████████████████| 20000/20000 [6:08:17<00:00,  1.10s/it]


In [6]:
for index, row in tqdm(en_items.iloc[20010:40010].iterrows(), total=20000):
    emotion = detect_emotion(row['excerpt_value_cleaned'])
    en_items.at[index, 'emotion_llama'] = emotion

100%|███████████████████████████████████████████████████████████████████████████| 20000/20000 [6:05:51<00:00,  1.10s/it]


In [7]:
for index, row in tqdm(en_items.iloc[40010:60010].iterrows(), total=20000):
    emotion = detect_emotion(row['excerpt_value_cleaned'])
    en_items.at[index, 'emotion_llama'] = emotion

100%|███████████████████████████████████████████████████████████████████████████| 20000/20000 [6:09:21<00:00,  1.11s/it]


In [15]:
for index, row in tqdm(en_items.iloc[60010:].iterrows()):
    emotion = detect_emotion(row['excerpt_value_cleaned'])
    en_items.at[index, 'emotion_llama'] = emotion

18863it [7:28:22,  1.43s/it]


In [17]:
en_items

,Unnamed: 0,id,graph,label,source_id,source_label,source_url,source_date,source_language,relevantExcerpt,...,words,year,sentiment,excerpt_value_cleaned,source_label_cleaned,geoname_id,countries,final_country_code,final_country_name,emotion_llama
item_id,,,,,,,,,,,,,,,,,,,,,
0,0,http://data.odeuropa.eu/smell/a09e1c3c-f023-5e...,http://data.odeuropa.eu/british-library,smell,http://data.odeuropa.eu/source/56c5cda7-750c-5...,Geschichte der Stadt und Bäder zu Baden [in C...,NaN,1880,en,http://data.odeuropa.eu/source/56c5cda7-750c-5...,...,"['sniffing', 'the party', 'betimes']",1880,0.088636,34 owd siah waved termed thick knotty meet meb...,geschichte der stadt und bäder zu baden canto...,10179318,['IR'],IR,Iran,fear
1,1,http://data.odeuropa.eu/smell/a09e1c3c-f023-5e...,http://data.odeuropa.eu/british-library,smell,http://data.odeuropa.eu/source/56c5cda7-750c-5...,Geschichte der Stadt und Bäder zu Baden [in C...,NaN,1880,en,http://data.odeuropa.eu/source/56c5cda7-750c-5...,...,"['Bartlett', 'perfume', 'scent', 'very odorous...",1880,-0.071652,179 arm linked lucy phœbe louise following lit...,geschichte der stadt und bäder zu baden canto...,10179318,['IR'],IR,Iran,joy
2,2,http://data.odeuropa.eu/smell/a09e1c3c-f023-5e...,http://data.odeuropa.eu/british-library,smell,http://data.odeuropa.eu/source/56c5cda7-750c-5...,Geschichte der Stadt und Bäder zu Baden [in C...,NaN,1880,en,http://data.odeuropa.eu/source/56c5cda7-750c-5...,...,"['stench', 'noysome', 'his', 'crawling']",1880,-0.062500,104 picture eccho forth grones hollow truncke ...,geschichte der stadt und bäder zu baden canto...,10179318,['IR'],IR,Iran,fear
3,3,http://data.odeuropa.eu/smell/a09e1c3c-f023-5e...,http://data.odeuropa.eu/british-library,smell,http://data.odeuropa.eu/source/56c5cda7-750c-5...,Geschichte der Stadt und Bäder zu Baden [in C...,NaN,1880,en,http://data.odeuropa.eu/source/56c5cda7-750c-5...,...,['fragrant'],1880,0.083333,nothing perhaps sends quicksilver sphits highe...,geschichte der stadt und bäder zu baden canto...,10179318,['IR'],IR,Iran,anticipation
4,4,http://data.odeuropa.eu/smell/a09e1c3c-f023-5e...,http://data.odeuropa.eu/british-library,smell,http://data.odeuropa.eu/source/56c5cda7-750c-5...,Geschichte der Stadt und Bäder zu Baden [in C...,NaN,1880,en,http://data.odeuropa.eu/source/56c5cda7-750c-5...,...,"['scent', 'thy', 'so much sharper']",1880,0.202273,open space sent forth shout joy echoed mountai...,geschichte der stadt und bäder zu baden canto...,10179318,['IR'],IR,Iran,joy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78868,78868,http://data.odeuropa.eu/smell/a9ec50f3-cdc7-5f...,http://data.odeuropa.eu/medical-heritage,smell,http://data.odeuropa.eu/source/5b3e4d67-69f1-5...,"The Jordan and the Rhine : or, The East and th...",http://archive.org/details/b29352733,1854,en,http://data.odeuropa.eu/source/5b3e4d67-69f1-5...,...,"['herbs', 'scented', 'odours', 'Sweetly scente...",1854,0.350000,ever sweeter bloomingfieldsandverdantmeadows o...,jordan rhine east west result five year reside...,9062341,['LB'],LB,Lebanon,anticipation
78869,78869,http://data.odeuropa.eu/smell/a9ec50f3-cdc7-5f...,http://data.odeuropa.eu/medical-heritage,smell,http://data.odeuropa.eu/source/5b3e4d67-69f1-5...,"The Jordan and the Rhine : or, The East and th...",http://archive.org/details/b29352733,1854,en,http://data.odeuropa.eu/source/5b3e4d67-69f1-5...,...,"['smell', 'like the smell', 'The dying patriar...",1854,0.500000,â   5th dying patriarch isaac pleased smell ...,jordan rhine east west result five year reside...,9062341,['LB'],LB,Lebanon,trust
78870,78870,http://data.odeuropa.eu/smell/a9ec50f3-cdc7-5f...,http://data.odeuropa.eu/medical-heritage,smell,http://data.odeuropa.eu/source/5b3e4d67-69f1-5...,"The Jordan and the Rhine : or, The East and th...",http://archive.org/details/b29352733,1854,en,http://data.odeuropa.eu/source/5b3e4d67-69f1-5...,...,['intolerable'],1854,0.025000,mercy god ex hausted sin may appear 

In [101]:
for index, row in en_items[en_items['emotion_llama'].isnull()].iterrows():
    emotion = detect_emotion(row['excerpt_value_cleaned'])
    en_items.at[index, 'emotion_llama'] = emotion

In [125]:
en_items['emotion_llama'].isnull().sum()

0

In [103]:
# null_emotion_llama = en_items[en_items['emotion_llama'].isnull()]

# Save the filtered rows to a CSV file
null_emotion_llama.to_csv('null_emotion_llama.csv', index=False)

In [100]:
import pandas as pd
import re
import unicodedata

null_emotion_llama = en_items[en_items['emotion_llama'].isnull()]

def clean_text(text):
    if pd.isna(text):
        return text
    text = unicodedata.normalize('NFKD', text)
    text = ''.join(c for c in text if not unicodedata.combining(c))
    text = re.sub(r'[^a-zA-Z0-9\s.,!?\'"]', '', text)
    return text

null_emotion_llama['excerpt_value_cleaned'] = null_emotion_llama['excerpt_value_cleaned'].apply(clean_text)

en_items.loc[en_items['emotion_llama'].isnull(), 'excerpt_value_cleaned'] = null_emotion_llama['excerpt_value_cleaned']

/var/folders/jr/qdhrcycd4rx8yxrqjzg8czdw0000gn/T/ipykernel_18913/2117363855.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_emotion_llama['excerpt_value_cleaned'] = null_emotion_llama['excerpt_value_cleaned'].apply(clean_text)


In [118]:
def detect_emotion_ext(text):
    """Send text to the local LLaMA model and return the detected emotion based on Plutchik's wheel."""
    emotions = ["joy", "trust", "fear", "surprise", "sadness", "disgust", "anger", "anticipation"]
    pattern = re.compile(r'\b(' + '|'.join(emotions) + r')\b', re.IGNORECASE)
    
    data = {
        "model": "llama3",
        "prompt": f"Your answer can be only one out of these 8: [joy, trust, fear, surprise, sadness, disgust, anger and anticipation]. You are trained to identify emotions based on Plutchiks wheel. You will only answer with the primary emotion and not a single word more. Your only options are: joy, trust, fear, surprise, sadness, disgust, anger and anticipation. What is the primary emotion in this text based on Plutchiks wheel of emotions: {text}",
        "stream": False
    }
    
    try:
        response = requests.post('http://localhost:11434/api/generate', json=data)
        response_data = response.json()
        response_text = response_data["response"].strip().lower()
        print(response_text)
        
        match = pattern.search(response_text)
        if match:
            return match.group(0)
        else:
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None


In [126]:
for index, row in en_items[en_items['emotion_llama'].isnull()].iterrows():
    print(index)
    emotion = detect_emotion_ext(row['excerpt_value_cleaned'])
    en_items.at[index, 'emotion_llama'] = emotion

In [127]:
en_items.to_csv('full_with_llama_emotions.csv')